# Text summarization with T5 on XSum

We are going to fine-tune [T5 implemented by HuggingFace](https://huggingface.co/t5-small) for text summarization on the [Extreme Summarization (XSum)](https://huggingface.co/datasets/xsum) dataset.
The data if composed by news articles and the corresponding summaries.

This notebooks is based on the script [run_summarization_no_trainer.py](https://github.com/huggingface/transformers/blob/v4.12.5/examples/pytorch/summarization/run_summarization_no_trainer.py) from HuggingFace.

We will be using the following model sizes available from HuggingFace

| Variant                                     |   Parameters    |
|:-------------------------------------------:|----------------:|
| [T5-small](https://huggingface.co/t5-small) |    60,506,624   | 
| [T5-large](https://huggingface.co/t5-large) |   737,668,096   | 
| [T5-3b](https://huggingface.co/t5-3b)       | 2,851,598,336   | 




More info:
* [T5 on HuggingFace docs](https://huggingface.co/transformers/model_doc/t5.html)

In [1]:
import os
import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader

In [2]:
hf_model = 't5-small'
t5_cache = os.path.join(os.getcwd(), 'cache')

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    hf_model,
    use_fast=True,
    cache_dir=os.path.join(t5_cache, f'{hf_model}_tokenizer')
)

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    hf_model,
    cache_dir=os.path.join(t5_cache, f'{hf_model}_model')
)

In [8]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in parameters])
print(f'\n {num_params:,} parameters\n')
parameters = filter(lambda p: p.requires_grad, model.parameters())


 60,506,624 parameters



In [6]:
raw_datasets = load_dataset('xsum')

Using custom data configuration default
Reusing dataset xsum (/users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
max_source_length = 1024
max_target_length = 128
ignore_pad_token_for_loss = True
padding = False            # else 'max_length'
label_pad_token_id = -100  # else tokenizer.pad_token_id
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

In [9]:
def preprocess_function(examples, text_column='document',
                        summary_column='summary', prefix='summarize: '):
    inputs = examples[text_column]
    targets = examples[summary_column]
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=max_source_length,
                             padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length,
                           padding=padding, truncation=True)

    # padding in the loss.
    if padding == "max_length" and ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100)
             for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id
)

processed_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=per_device_train_batch_size
)

eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=data_collator,
    batch_size=per_device_eval_batch_size
)

Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-8909393d709d88b1.arrow
Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-579c7d5bb0d337f2.arrow
Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-24a96f003ada9700.arrow


In [14]:
weight_decay = 0.0
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters()
                   if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters()
                   if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)

In [16]:
device = 0
model.to(device)
model.train()

model.training

True

In [17]:
for step, batch in enumerate(train_dataloader):  
    optimizer.zero_grad()
    outputs = model(**batch.to(device))
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    
    # stop after 10 steps for the demo:
    if step > 10:
        break

## Evaluation

In [18]:
metric = load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [19]:
model.eval()

gen_kwargs = {
    "max_length": 128,   # args.val_max_target_length if args is not None else config.max_length,
    "num_beams": None,   # args.num_beams,
}
for step, batch in enumerate(eval_dataloader):
    if step > 20:
        break

    with torch.no_grad():
        generated_tokens = model.generate(
            batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            # **gen_kwargs,
        )

        labels = batch["labels"]
        generated_tokens = generated_tokens.cpu().numpy()
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        print('pred:', decoded_preds)
        print('labels:', decoded_labels)
        print('')

        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

result = metric.compute(use_stemmer=True)
# Extract a few results from ROUGE
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

result = {k: round(v, 4) for k, v in result.items()}

print(result)

pred: ["apple's lawyers said they were willing to publish a clarification.\nthe company does not", 'no international side has toured Bangladesh since 20 people were killed in a siege in D', 'the premier is in the country for a three-day visit.\nhe is in', 'the subsea power hub (SPH) is being tested by Aberdeen based']
labels: ['US technology firm Apple has offered to refund Australian customers who felt misled about the 4G capabilities of the new iPad.', "England one-day captain Eoin Morgan and opening batsman Alex Hales have opted out of October's tour of Bangladesh because of security concerns.", 'Irish Taoiseach (PM) Leo Varadkar has engaged in some "sock diplomacy" in his first meeting with Canadian Prime Minister Justin Trudeau in Dublin.', 'A new ocean current energy system has been successfully deployed, according to the company behind it.']

pred: ['the last quarter saw insolvencies rise by 17.3% year-on-year to 2,8', "china's pledge comes ahead of talks in Paris to seek a new 